In [7]:
import boto3
from botocore.exceptions import ClientError
import pandas as pd
import base64
from spdynamodb import DynamoTable
from datetime import datetime
import json
import time
from decimal import Decimal

In [2]:
dt = DynamoTable()
try:
    dt.select_table('SkiLifts')
    print(dt)
except:
    dt.create_table(
        table_name='SkiLifts',
        partition_key='Lift',
        partition_key_type='S',
        sort_key="Metadata",
        sort_key_type="S"
)

- Table name: SkiLifts            
- Table arn: arn:aws:dynamodb:us-east-1:089715336747:table/SkiLifts            
- Table creation: 2023-05-28 19:18:27            
- [{'AttributeName': 'Lift', 'KeyType': 'HASH'}, {'AttributeName': 'Metadata', 'KeyType': 'RANGE'}]            
- [{'AttributeName': 'Lift', 'AttributeType': 'S'}, {'AttributeName': 'Metadata', 'AttributeType': 'S'}]            
- Point-in-time recovery status: DISABLED  |  Delete protection: False


In [7]:
dt.load_json("ski.json")

**TABLE:**
<br>

![SkiResortDataModel.png](images/ski_data01.png)

In [30]:
# Policy 1 - Allow access to all items in the table if the user is authenticated and primary key is Lift 23
policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AllowAccessToOnlyItemsMatchingUserID",
            "Effect": "Allow",
            "Action": [
                "dynamodb:GetItem",
                "dynamodb:BatchGetItem",
                "dynamodb:Query",
                "dynamodb:PutItem",
                "dynamodb:UpdateItem",
                "dynamodb:DeleteItem",
                "dynamodb:BatchWriteItem"
            ],
            "Resource": "arn:aws:dynamodb:us-east-1:089715336747:table/SkiLifts",
            "Condition": {
                "ForAllValues:StringEquals": {
                    "dynamodb:LeadingKeys": [
                        "Lift 23"
                    ]
                }
            }
        },
        {
            "Sid": "VisualEditor1",
            "Effect": "Allow",
            "Action": "dynamodb:ListTables",
            "Resource": "*"
        }
    ]
}
policy_name = 'dynamodb-policy-test-sample'

In [5]:
# Policy N°2 - Allow access to all items in the table if the user is authenticated and primary 
# key is Lift 23 and attributes are Lift, Metadata and Item
policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AllowAccessToOnlyItemsMatchingUserID",
            "Effect": "Allow",
            "Action": [
                "dynamodb:GetItem",
                "dynamodb:BatchGetItem",
                "dynamodb:Query",
                "dynamodb:PutItem",
                "dynamodb:UpdateItem",
                "dynamodb:DeleteItem",
                "dynamodb:BatchWriteItem"
            ],
            "Resource": "arn:aws:dynamodb:us-east-1:089715336747:table/SkiLifts",
            "Condition": {
                "ForAllValues:StringEquals": {
                    "dynamodb:LeadingKeys": [
                        "Lift 23"
                    ],
                "dynamodb:Attributes": [
                    "Lift",
                    "Metadata",
                    "Item"
                    ]
                },
                "StringEqualsIfExists": {
                    "dynamodb:Select": "SPECIFIC_ATTRIBUTES"
                }
            }
        },
        {
            "Sid": "VisualEditor1",
            "Effect": "Allow",
            "Action": "dynamodb:ListTables",
            "Resource": "*"
        }
    ]
}
policy_name = 'dynamodb-policy-test-sample'

In [10]:
# Policy N°3 - Allow access to especific table
policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "ListAndDescribe",
            "Effect": "Allow",
            "Action": [
                "dynamodb:List*",
                "dynamodb:DescribeReservedCapacity*",
                "dynamodb:DescribeLimits",
                "dynamodb:DescribeTimeToLive"
            ],
            "Resource": "*"
        },
        {
            "Sid": "SpecificTable",
            "Effect": "Allow",
            "Action": [
                "dynamodb:BatchGet*",
                "dynamodb:DescribeStream",
                "dynamodb:DescribeTable",
                "dynamodb:Get*",
                "dynamodb:Query",
                "dynamodb:Scan",
                "dynamodb:BatchWrite*",
                "dynamodb:CreateTable",
                "dynamodb:Delete*",
                "dynamodb:Update*",
                "dynamodb:PutItem"
            ],
            "Resource": "arn:aws:dynamodb:us-east-1:089715336747:table/SkiLifts"
        }
    ]
}
policy_name = 'dynamodb-policy-test-sample'

In [2]:
# Update policy or create if it doesn't exist
import boto3
from botocore.exceptions import ClientError
import json
iam = boto3.client('iam')
try:
    response = iam.create_policy(
        PolicyName=policy_name,
        PolicyDocument=json.dumps(policy)
    )
    print(f"Policy {policy_name} created successfully. Arn: {response['Policy']['Arn']}")

except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        #Update policy
        account_id = boto3.client('sts').get_caller_identity().get('Account')
        response = iam.get_policy(PolicyArn=f"arn:aws:iam::{account_id}:policy/{policy_name}")
        policy_arn = response['Policy']['Arn']
        # Delete all policy versions except the default version
        response = iam.list_policy_versions(PolicyArn=f"arn:aws:iam::{account_id}:policy/{policy_name}")
        for version in response['Versions']:
            if not version['IsDefaultVersion']:
                response = iam.delete_policy_version(
                    PolicyArn=policy_arn,
                    VersionId=version['VersionId']
                )   
        response = iam.create_policy_version(
            PolicyArn=policy_arn,
            PolicyDocument=json.dumps(policy),
            SetAsDefault=True
        )
        print(f"Policy {policy_name} updated successfully. Arn: {policy_arn}")
    else:
        print("Unexpected error: %s" % e)
    

Policy dynamodb-policy-test-sample updated successfully. Arn: arn:aws:iam::089715336747:policy/dynamodb-policy-test-sample


In [11]:
import boto3
from botocore.exceptions import ClientError
session = boto3.Session(profile_name='089715336747_DynamoAttributes')
dynamo_client = session.client('dynamodb', region_name='us-east-1')

### Policy N° 1

In [9]:
try:
    response = dynamo_client.get_item(TableName='SkiLifts', Key={'Lift': {'S': 'Lift 20'}, 'Metadata': {'S': 'Static Data'}})
    print(response.get('Item'))
except ClientError as e:
    print("Error:", e.response['Error']['Code'])

None


In [5]:
dynamo_client.get_item(TableName='SkiLifts', Key={'Lift': {'S': 'Lift 23'}, 'Metadata': {'S': 'Static Data'}})

{'Item': {'VerticalFeet': {'N': '900'},
  'Metadata': {'S': 'Static Data'},
  'LiftTime': {'S': '5:45'},
  'ExperiencedRidersOnly': {'BOOL': True},
  'Lift': {'S': 'Lift 23'}},
 'ResponseMetadata': {'RequestId': 'O057TNT9KN8EKRSN3M83H2LQFJVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Mon, 29 May 2023 16:55:32 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '153',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'O057TNT9KN8EKRSN3M83H2LQFJVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '3713276519'},
  'RetryAttempts': 0}}

### Policy N° 2

In [7]:
try:
    response = dynamo_client.get_item(TableName='SkiLifts', Key={'Lift': {'S': 'Lift 23'}, 'Metadata': {'S': 'Static Data'}})
    print(response.get('Item'))
except ClientError as e:
    print("Error:", e.response['Error']['Code'])

{'VerticalFeet': {'N': '900'}, 'Metadata': {'S': 'Static Data'}, 'LiftTime': {'S': '5:45'}, 'ExperiencedRidersOnly': {'BOOL': True}, 'Lift': {'S': 'Lift 23'}}


In [8]:
dynamo_client.get_item(TableName='SkiLifts', Key={'Lift': {'S': 'Lift 23'}, 'Metadata': {'S': 'Static Data'}}, AttributesToGet=['Lift', 'Metadata', 'Item'])

{'Item': {'Metadata': {'S': 'Static Data'}, 'Lift': {'S': 'Lift 23'}},
 'ResponseMetadata': {'RequestId': '9A0I91H0J10S3I6GUROG3944SFVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Mon, 29 May 2023 17:28:38 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '64',
   'connection': 'keep-alive',
   'x-amzn-requestid': '9A0I91H0J10S3I6GUROG3944SFVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '3339958042'},
  'RetryAttempts': 0}}

### Policy N° 3

In [12]:
dynamo_client.get_item(TableName='SkiLifts', Key={'Lift': {'S': 'Lift 23'}, 'Metadata': {'S': 'Static Data'}}, AttributesToGet=['Lift', 'Metadata', 'Item'])

{'Item': {'Metadata': {'S': 'Static Data'}, 'Lift': {'S': 'Lift 23'}},
 'ResponseMetadata': {'RequestId': 'EQ89AOBA5AP97EN235C9PTIEANVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Mon, 29 May 2023 17:30:22 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '64',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'EQ89AOBA5AP97EN235C9PTIEANVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '3339958042'},
  'RetryAttempts': 0}}

In [13]:
dynamo_client.get_item(TableName='Bookks', Key={'Author': {'S': 'William Shakespeare'}, 'Title': {'S': 'The Tempest'}})

ClientError: An error occurred (AccessDeniedException) when calling the GetItem operation: User: arn:aws:sts::089715336747:assumed-role/AWSReservedSSO_DynamoAttributes_be053d6f391d2619/TestUser is not authorized to perform: dynamodb:GetItem on resource: arn:aws:dynamodb:us-east-1:089715336747:table/Bookks because no identity-based policy allows the dynamodb:GetItem action